In [1]:
1 + 1

2

In [2]:
dev_mode = false

using Distributed

if !dev_mode
    if nprocs() == 1 addprocs(4) end
end

using MLToolkit: flatten_dict, dict2namedtuple

@everywhere begin 
    using Random: seed!
    using Revise
    using RMMMDNets
end

;

┌ Info: Recompiling stale cache file /afs/inf.ed.ac.uk/user/s16/s1672897/.julia/compiled/v1.2/RMMMDNets/mJmbq.ji for RMMMDNets [7ea7fd3b-14e3-4223-94d2-c341911ad28e]
└ @ Base loading.jl:1240


In [3]:
# dataset = "gaussian"
dataset = "ring"
# dataset = "mnist"

# model_name = "mmdnet"
model_name = "rmmmdnet"

include("scripting.jl")

args_list = if dev_mode
    args = get_args(
        dataset, 
        model_name;
        override=Dict{Symbol,Any}(), suffix=""
    )
    [args]
else
    let d=dataset, m=model_name
        [get_args(d, m; override=Dict(:seed => seed), suffix="seed=$seed") for seed in [1, 2, 3, 4]]
    end
end

;

┌ Info: On dataset ring with args
│   batch_size_gen = 500
│   batch_size = 500
│   Df_h = 100,10
│   Dg_h = 100,100
│   σ_last = identity
│   D_z = 10
│   D_fx = 1
│   dataset = ring
│   σ = relu
│   opt = adam_akash
│   model_name = rmmmdnet
│   n_epochs = 50
│   σs = 1
│   lr = 0.001
│   seed = 1
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/RMMMDNets.jl/scripting.jl:84
┌ Info: On dataset ring with args
│   batch_size_gen = 500
│   batch_size = 500
│   Df_h = 100,10
│   Dg_h = 100,100
│   σ_last = identity
│   D_z = 10
│   D_fx = 1
│   dataset = ring
│   σ = relu
│   opt = adam_akash
│   model_name = rmmmdnet
│   n_epochs = 50
│   σs = 1
│   lr = 0.001
│   seed = 2
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/RMMMDNets.jl/scripting.jl:84
┌ Info: On dataset ring with args
│   batch_size_gen = 500
│   batch_size = 500
│   Df_h = 100,10
│   Dg_h = 100,100
│   σ_last = identity
│   D_z = 10
│   D_fx = 1
│   dataset = ring
│   σ = relu
│   opt = adam_akash
│   model_name

In [4]:
@everywhere function run_exp(args)
    data = get_data(args.dataset)
    
    seed!(args.seed)

    model = get_model(args, data)

    vis(data, model)

    dataloader = DataLoader(data, args.batch_size)
    
    train!(model, args.n_epochs, dataloader)
end

@sync @distributed for args in args_list
    run_exp(args)
end

;

┌ Info: Init rmmmdnet with 12,723 parameters
└   logdir = "ring/rmmmdnet/opt=adam_akash-σ=relu-σs=1-Df_h=100,10-Dg_h=100,100-D_fx=1-D_z=10-seed=3/21092019-15-42-16"
┌ Info: Init rmmmdnet with 12,723 parameters
└   logdir = "ring/rmmmdnet/opt=adam_akash-σ=relu-σs=1-Df_h=100,10-Dg_h=100,100-D_fx=1-D_z=10-seed=1/21092019-15-42-16"
┌ Info: Init rmmmdnet with 12,723 parameters
└   logdir = "ring/rmmmdnet/opt=adam_akash-σ=relu-σs=1-Df_h=100,10-Dg_h=100,100-D_fx=1-D_z=10-seed=2/21092019-15-42-16"
┌ Info: Init rmmmdnet with 12,723 parameters
└   logdir = "ring/rmmmdnet/opt=adam_akash-σ=relu-σs=1-Df_h=100,10-Dg_h=100,100-D_fx=1-D_z=10-seed=4/21092019-15-42-16"


Progress: 100%|█████████████████████████████████████████| Time: 0:02:08
Progress: 100%|█████████████████████████████████████████| Time: 0:02:08
Progress: 100%|█████████████████████████████████████████| Time: 0:02:11
Progress: 100%|█████████████████████████████████████████| Time: 0:02:11
